## Example interactive dqdv session

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from cellpy import (cellreader, prms, log)

In [ ]:
import bokeh
from bokeh.io import output_notebook
output_notebook()

In [ ]:
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
my_data = cellreader.CellpyData()
my_data.load(filename)

In [ ]:
from cellpy.utils import ica

In [ ]:
from bokeh.plotting import figure, show
import bokeh.palettes

In [ ]:
# ica.dqdv??

In [ ]:
# my_data.get_cap?

In [ ]:
cycle_df = my_data.get_cap(1, categorical_column=True, method = "forth")
plot = figure(plot_width=800, plot_height=300)
plot.line(x=cycle_df.capacity, y=cycle_df.voltage)
show(plot)

In [ ]:
cycles = range(2,18)
colors = bokeh.palettes.cividis(len(cycles))
colors = bokeh.palettes.viridis(len(cycles))
#colors = bokeh.palettes.gray(len(cycles)+10)

del plot
plot = figure(plot_width=800, plot_height=600)

cycle_df = my_data.get_cap(1, categorical_column=True, method = "forth-and-forth")
v, dq = ica.dqdv_cycle(cycle_df)
plot.line(x=v, y=dq, line_color='red', legend='cycle 1')

for n, cycyno in enumerate(cycles):
    color = colors[n]
    name = f"cycle {cycyno}"
    #c1, v1 = my_data.get_ccap(cycyno)
    #c2, v2 = my_data.get_dcap(cycyno)
    #v_charge, dq_charge = ica.dqdv(v1, c1)
    #v_discharge, dq_discharge = ica.dqdv(v2, c2)

    cycle_df = my_data.get_cap(cycyno, categorical_column=True, method = "forth-and-forth")
    v, dq = ica.dqdv_cycle(cycle_df)

    plot.line(x=v, y=dq, line_color=color, legend=name)

    
plot.legend.location = "top_right"
plot.legend.click_policy="hide"

show(plot)